In [1]:
import tensorflow as tf
import numpy as np
import os
#畳み込み層の実装
def conv_layer(input_tensor,name,n_output_channels,ksize,padding='SAME',strides=[1,1,1,1]):
    with tf.variable_scope(name):
        n_input_channels=1
        
        weights_shape=(list(ksize)+[n_input_channels,n_output_channels])
        weights=tf.get_variable(name='_weights',shape=weights_shape)
        
        print(weights)
        
        biases=tf.get_variable(name='_biases',initializer=tf.zeros(shape=[n_output_channels]))
        
        print(biases)
        #重みと入力テンソルの掛け算
        conv=tf.nn.conv2d(input=input_tensor,filter=weights,strides=strides,padding=padding)
        print(conv)
        #重みと入力テンソルの積にバイアスを加える
        conv=tf.nn.bias_add(conv,biases,name='net_pre_activation')
        print(conv)
        #線形活性化
        conv=tf.nn.relu(conv,name='activation')
        print(conv)
        
        return conv        

In [2]:
#全結合層の実装
def fc_layer(input_tensor,name,n_output_units,activation_fn=None):
    with tf.variable_scope(name):
        input_shape=input_tensor.get_shape().as_list()[1:]
        n_input_units=np.prod(input_shape)
        
        if len(input_shape)>1:
            input_tensor=tf.reshape(input_tensor,shape=(-1,n_input_units))
            
        weights_shape=[n_input_units,n_output_units]
        weights=tf.get_variable(name='__weights',shape=weights_shape)
        print(weights)
        
        biases=tf.get_variable(name='__biases',initializer=tf.zeros(shape=[n_output_units]))
        print(biases)
        #入力テンソルと重みの積の総和
        layer=tf.matmul(input_tensor,weights)
        print(layer)
        #バイアスが入る
        layer=tf.nn.bias_add(layer,biases,name='net_pre_activation')
        print(layer)
        
        if activation_fn is None:
            return layer
        layer=activation_fn(layer,name='activation')
        print(layer)
        return layer

In [2]:
#model構築
def build():
    #784のところは学習させる画像のピクセル数による
    #画像を学習させるときは一次元配列に直す必要性あり
    tf_x=tf.placeholder(tf.float32,shape=[None,784],name='tf_x')
    tf_y=tf.placeholder(tf.int32,shape=[None],name='tf_y')
    
    #入力テンソルはバッチ学習ができるように四次元テンソルにしておく
    tf_x_img=tf.reshape(tf_x,shape=[-1,int(np.sqrt()),int(np.sqrt()),1],name='tf_x_reshape')
    tf_y_onehot=tf.one_hot(indices=tf_y,depth=8,dtype=tf.float32,name='tf_y_onehot')
    
    #第一層
    print('==building 1st layer==')
    h1=conv_layer(tf_x_img,name='conv_1',n_output_channels=16,ksize=(6,6),padding='VALID')
    
    print('==h1 pooling==')
    h1_pool=tf.nn.max_pool(h1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    #第二層
    print('==building 2nd layer==')
    h2=conv_layer(h1_pool,name='conv_2',n_output_channels=32,ksize=(6,6),padding='VALID')
    
    print('==h2 pooling==')
    h2_pool=tf.nn.max_pool(h2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    #第三層
    print('==building 3rd layer==')
    h3=conv_layer(h2_pool,name='conv_3',n_output_channels=64,ksize=(4,4),padding='VALID')
    
    print('==h3 pooling==')
    h3_pool=tf.nn.max_pool(h3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    #第四層
    h4=fc_layer(h3_pool,name='linear_4',n_output_units=4096,activation_fn=tf.nn.relu)
    
    #dropout
    keep_prob=tf.placeholder(tf.float32,name='tf_keep_prob')
    h4_drop=tf.nn.dropout(h3_pool,keep_prob=keep_prob,name='dropout_layer')
    
    #第五層
    #classの数がn_output_units 
    #class数はアーテイストの数による
    h5=fc_layer(h4_drop,name='linear_5',n_output_units=7,activation_fn=None)
    
    
    #予測とラベル
    predictions={'probabilities':tf.nn.softmax(h5,name='probabilities'),'labels':
                tf.cast(tf.argmax(h5,axis=1),tf.int32,name='labels')}
    
    #損失関数、コスト
    cost=tf.reduce_mean(tf.softmax_cross_entropy_with_logits(logits=h5,labels=tf_y_onehot),name=(cross_entropy_loss))
    
    #最適化
    optimizer=tf.train.AdamOptimizer(learning_rate)
    optimizer=optimizer.minimize(cost,name='train_op')
    
    #予測正解率を特定
    correct_predictions=tf.eqaul(predictions['labels'],tf_y,name='correct_pred')
    
    accuracy=tf.reduce_mean(tf.cast(correct_predictions,tf.float32),name='accuracy')
    
    
    
    
    

In [3]:
"""train:学習をするアルゴリズムを構築する関数
   save:学習したモデルを保存する関数
   load:学習したモデルを呼び出す関数
   predict:テストデータと比較する関数"""

'train:学習をするアルゴリズムを構築する関数\n   save:学習したモデルを保存する関数\n   load:学習したモデルを呼び出す関数\n   predict:テストデータと比較する関数'

In [ ]:
#バッチ学習
def batch_generator(X_train,y_train,shuffle=True,batch_size=64,random_seed=None):
    idx=np.arange(y_train.shape[0])
    
    if shuffle:
        np.random.RandomState(random_state).shuffle(idx)
        X=X_train[idx]
        y=y_train[idx]
        
        
    for i in range(0,X.shape[0],batch_size):
        yield(X[i*i+batch_size,:],y[i:i+batch_size])

In [4]:
#train
def train(sess,training_set,validation_set=None,initialize=True,epochs=20,
          shuffle=True,dropout=0.5,random_seed=None):
    #training dataの成型
    X_train=np.array(training_set[0])
    y_train=np.array(training_set[1])
    
    #training_lossを評価
    training_loss=[]
    
    if initialize:
        sess.run(tf.global_variables_initializer())
        
    #バッチ学習
    np.random.seed(random_seed)
    
    
    #学習スタート
    for epoch in range(epochs):
        bach_gen=batch_generator(X_train,y_train,shuffle=shuffle)
        avg_loss=0.0
        
        for idx,(batch_x,batch_y) in enumerate(batch_gen):
            feed={'tf_x:0':batch_x,'tf_y:0':batch_y,'tf_keep_prob:0':dropout}
            loss,_=sess.run(['cost:0','train_op'],feed_dict=feed)
            avg_loss+=loss
            
        training_loss.append(avg_loss/(idx+1))
        print('epoch%d training_loss:%.7f'%(epoch,avg_loss))
        
        #検証データ、検証するとき
        if validation_set is not None:
            feed={'tf_x:0':validation_set[0],'tf_y:0':validation_set[1],'tf_keep_prob:0':1.0}
            
            valid_acc=sess.run('accuracy:0',feed_dict=feed)
            
            print('validation acc%.5f'%(valid_acc))
            
        else:
            print()
            
#save
def save(saver,sess,epoch,path='./model/'):
    #saverのインスタンスはsaver=tf.train.saver()
    if not os.path.isdir(path):
        os.mkdir(path)
    
    print('saving_model in &s'%path)
    saver.save(sess,os.path.join(path,'cnn-model.ckpt'),global_step=epoch)
    
def load(saver,sess,path,epoch):
    print('loading model from %s'%path)
    saver.restore(sess,os.path.join(path,'cnn-model.ckpt-%d'%epoch))
    
#predict
#return_probaは確率的スコアリングをするかしないかを決める
def predict(sess,X_test,return_proba=False):
    feed={'tf_x:0':X_test,'tf_keep_prob:0':1.0}
    if return_proba:
        return sess.run('probabilities:0',feed_dict=feed)
    else:
        return sess.run('labels:0',feed_dict=feed)
    
    
    
        

In [ ]:
#音楽の曲調をケプストラム分析で抽出し、画像データに変換してデータセットにする
import numpy as np
import librosa
from scipy.io.wavefile import read

def __init__(self,wavfile,chunk,length,expected_fs=None):
    fs,all_data=read(wavfile)
    if expected_fs!=None and expected_fs!=fs:
        print('It has difference between expected_fs and fs')
        raise AssertionError
    
    #dataの形を成型
    all_data=all_data.astype('float64')-128.0
    all_data/=128.0
    
    self.data=all_data
    self.sampling_rate=fs
    self.chunk=chunk
    self.length=length
    
    #noise読み込み
    self.noise=np.load('noise/noise.npy')
    self.noise=self.noise.astype('float32')/128.0
    
    
    #index??
    n_blocks_all=len(all_data)-self.chunk*self.length-1
    self.indexes=np.linspace(0,n_blocks_all,int(n_blocks_all/5.0))
    self.n_blocks=len(self.indexes)
    
    print('sampling_rate:{}'.format(fs))
    
    
def shuffle_indexes(self):
    self.indexes=np.random.permutation(len(self.indexes))
    
#ノイズを追加
def add_noise(self,data,scale=None):
    
    
    
    
    
    